# Лабораторная работа 2. Вероятностные модели.

Результат лабораторной работы − отчет. Мы предпочитаем принимать отчеты в формате ноутбуков IPython (ipynb-файл). Постарайтесь сделать ваш отчет интересным рассказом, последовательно отвечающим на вопросы из заданий. Помимо ответов на вопросы, в отчете также должен быть код, однако чем меньше кода, тем лучше всем: нам − меньше проверять, вам — проще найти ошибку или дополнить эксперимент. При проверке оценивается четкость ответов на вопросы, аккуратность отчета и кода.

Мы уверены, что выполнение лабораторных работ занимает значительное время, поэтому не рекомендуем оставлять их на последний вечер перед сдачей.

### Оценивание и штрафы
* Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи)
* Максимально допустимая оценка за работу — 15 баллов. Также в результате выполнения заданий у вас получится решение [задачи конкурса](https://www.kaggle.com/c/competition-2-yandex-shad-spring-2021), которое после небольшой доработки принесёт ещё 5 баллов за пробитие Medium Baseline.
* Сдавать задание после указанного срока сдачи нельзя
* «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов и понижают карму (подробнее о плагиате см. на странице курса)
* Если вы нашли решение какого-то из заданий в открытом источнике, необходимо прислать ссылку на этот источник (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник)
* Не оцениваются задания с удалёнными формулировками
* Не оценивается лабораторная работа целиком, если она была выложена в открытый источник

Обратите внимание, что мы не ставим оценку за просто написанный код, корректная работоспособность которого не подтверждена экспериментами.


Цель этой лабораторной работы – научиться строить вероятностные модели и оптимизировать их параметры на примере задачи оценки риска заболевания сахарным диабетом. На задачу оценки риска болезни мы посмотрим со стороны страховой компании.

Если человек перестанет проходить обследования, и страховая так и не узнает, развился ли у него диабет, то и расходов, связанных с его заболеванием не будет, т.е. можно считать, что такой человек остался здоров. То же касается людей, у которых заболевание впервые обнаружат более чем через 5 лет.

Чтобы рассчитать математическое ожидание затрат на лечение клиента, страховая хочет получить в качестве результата работы модели непосредственно вероятность того, что у человека, не страдающего от заболевания, оно разовьётся в течение 5 лет. Поэтому в качестве метрики качества была выбрана бинарная кросс-энтропия (она же logloss) между предсказанными вероятностями и истинными метками классов:


$$\text{crossentropy}(y, p) = -\frac{1}{N}\sum_{i=1}^N\left[y_i\log(p_i) + (1-y_i)\log(1-p_i)\right]$$



**Задание 1** Препроцессинг (1 балл) 

- Прочитайте описание [набора данных и задачи конкурса 2](https://www.kaggle.com/c/competition-2-yandex-shad-spring-2021).
- Загрузите обучающий набор данных (X_train.csv, y_train.csv).
- Обратите внимание, что часть информации о клиентах неизвестна на момент заключения договора. Соответствующие признаки отсутствуют в X_test.csv.
- Заполните пропуски в данных. Для этого могут пригодиться методы из sklearn.impute или pandas.DataFrame.fillna.
- По желанию используйте любой препроцессинг данных, добавляйте новые признаки и т.п. ваша задача — добиться сходимости и высокого качества полученных моделей.
- Разбейте обучающую выборку на lab_train и lab_test, которые будете использовать для оценки всех построенных моделей в лабораторной работе. При желании использовать для оценки качества кросс-валидацию необходимо проконтролировать, чтобы для всех моделей использовались одни и те же разбиения.

In [15]:
import pandas as pd
from sklearn import impute
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

In [6]:
X_train = pd.read_csv('X_train.csv')
y_train = pd.read_csv('y_train.csv', index_col='Id')

In [3]:
only_train_col = ['year of followup', 'Fasting plasma glucose (final visit)', 'censor of diabetes at followup', 
                 'Diabetes diagnosed during followup']

In [7]:
X_train.drop(only_train_col, inplace=True, axis=1)

In [4]:
X_train = pd.get_dummies(X_train, columns=['Gender', 'site', 'smoking status', 'drinking status'])

In [14]:
y_test.isna().sum()/len(y_test)

Age                                      0.000000
Gender                                   0.000000
site                                     0.000000
height                                   0.000000
weight                                   0.000000
Body Mass Index                          0.000000
Systolic blood pressure                  0.000107
diastolic blood pressure                 0.000107
Fasting plasma glucose                   0.000000
Cholesterol                              0.023222
Triglyceride                             0.023338
High-density lipoprotein cholesterol     0.448150
Low-density lipoprotein cholesterol      0.442955
Alanine aminotransferase                 0.008450
Aspartate transaminase                   0.581608
Blood urea nitrogen                      0.102573
Creatinine clearance rate                0.053428
smoking status                           0.717060
drinking status                          0.717060
family history of diabetes               0.000000


In [23]:
smoking_data = pd.DataFrame(SimpleImputer(strategy='median').fit_transform(X_train[~X_train['smoking status'].isna()].copy()),
                            columns=X_train.columns)
smoking_target = smoking_data.pop('smoking status')

In [24]:
smoking_data.isna().sum()

Age                                      0
Gender                                   0
site                                     0
height                                   0
weight                                   0
Body Mass Index                          0
Systolic blood pressure                  0
diastolic blood pressure                 0
Fasting plasma glucose                   0
Cholesterol                              0
Triglyceride                             0
High-density lipoprotein cholesterol     0
Low-density lipoprotein cholesterol      0
Alanine aminotransferase                 0
Aspartate transaminase                   0
Blood urea nitrogen                      0
Creatinine clearance rate                0
Fasting plasma glucose (final visit)     0
Diabetes diagnosed during followup       0
censor of diabetes at followup           0
year of followup                         0
drinking status                          0
family history of diabetes               0
dtype: int6

In [26]:
smoking_clf = LogisticRegression(max_iter=5000, solver='liblinear')
smoking_clf.fit(smoking_data, smoking_target)

LogisticRegression(max_iter=5000, solver='liblinear')

In [10]:
X_train

,Age,height,weight,Body Mass Index,Systolic blood pressure,diastolic blood pressure,Fasting plasma glucose,Cholesterol,Triglyceride,High-density lipoprotein cholesterol,...,site_12,site_13,site_14,site_16,smoking status_1.0,smoking status_2.0,smoking status_3.0,drinking status_1.0,drinking status_2.0,drinking status_3.0
0,53,163.5,64.1,24.00,106.0,68.0,4.83,3.84,1.35,1.16,...,0,1,0,0,0,0,1,0,0,1
1,26,156.0,68.5,28.10,98.0,68.0,5.12,4.12,0.72,NaN,...,0,0,0,0,0,0,1,0,0,1
2,40,152.0,47.0,20.30,111.0,63.0,5.62,4.30,0.70,1.32,...,0,0,0,0,0,0,1,0,0,1
3,34,177.0,68.0,21.71,133.0,81.0,4.84,4.35,1.39,NaN,...,0,0,0,0,0,0,0,0,0,0
4,34,178.0,68.0,21.50,124.0,84.0,3.73,5.02,1.12,NaN,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,72,161.5,57.0,21.90,121.0,87.0,5.90,5.54,1.17,NaN,...,0,0,0,0,0,0,0,0,0,0
99996,51,177.5,83.0,26.30,123.0,79.0,4.76,5.70,2.15,NaN,...,0,0,0,0,0,0,0,0,0,0
99997,34,173.0,59.9,20.00,89.0,55.0,5.20,3.57,0.44,NaN,...,0,0,0,0,0,0,0,0,0,0
99998,32,187.0,96.0,27.50,147.0,104.0,4.77,4.22,1.02,NaN,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X_train.dropna()

In [11]:
replace_nan = SimpleImputer(strategy='median')
X_train = pd.DataFrame(replace_nan.fit_transform(X_train), columns=X_train.columns, index=X_train.index)

In [12]:
X_train.columns

Index(['Age', 'height', 'weight', 'Body Mass Index', 'Systolic blood pressure',
       'diastolic blood pressure', 'Fasting plasma glucose', 'Cholesterol',
       'Triglyceride', 'High-density lipoprotein cholesterol ',
       'Low-density lipoprotein cholesterol', 'Alanine aminotransferase',
       'Aspartate transaminase', 'Blood urea nitrogen',
       'Creatinine clearance rate', 'family history of diabetes', 'Gender_1',
       'Gender_2', 'site_0', 'site_2', 'site_3', 'site_4', 'site_5', 'site_6',
       'site_7', 'site_8', 'site_9', 'site_10', 'site_11', 'site_12',
       'site_13', 'site_14', 'site_16', 'smoking status_1.0',
       'smoking status_2.0', 'smoking status_3.0', 'drinking status_1.0',
       'drinking status_2.0', 'drinking status_3.0'],
      dtype='object')

In [13]:
scaler = StandardScaler()
scaler.fit(X_train.loc[:, :'Creatinine clearance rate'])
X_train_scaled = X_train.copy()
X_train_scaled.loc[:, :'Creatinine clearance rate'] = \
    scaler.transform(X_train.loc[:, :'Creatinine clearance rate'])

In [14]:
X_train_scaled

,Age,height,weight,Body Mass Index,Systolic blood pressure,diastolic blood pressure,Fasting plasma glucose,Cholesterol,Triglyceride,High-density lipoprotein cholesterol,...,site_12,site_13,site_14,site_16,smoking status_1.0,smoking status_2.0,smoking status_3.0,drinking status_1.0,drinking status_2.0,drinking status_3.0
0,0.861064,-0.357051,-0.050174,0.228036,-0.800849,-0.572614,-0.139734,-0.969961,0.016151,-0.860153,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-1.272461,-1.256006,0.309383,1.452767,-1.290199,-0.572614,0.334063,-0.656250,-0.595753,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
2,-0.166189,-1.735448,-1.447544,-0.877208,-0.495006,-1.034258,1.150955,-0.454578,-0.615178,-0.162992,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
3,-0.640306,1.261067,0.268524,-0.456021,0.850707,0.627662,-0.123396,-0.398559,0.055002,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.640306,1.380928,0.268524,-0.518751,0.300188,0.904649,-1.936895,0.352107,-0.207242,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2.362434,-0.596772,-0.630368,-0.399265,0.116682,1.181635,1.608414,0.934714,-0.158679,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,0.703025,1.320998,1.494287,0.915080,0.239019,0.443004,-0.254099,1.113977,0.793172,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,-0.640306,0.781625,-0.393388,-0.966823,-1.840718,-1.772889,0.464766,-1.272468,-0.867710,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,-0.798345,2.459674,2.556615,1.273538,1.707069,2.751226,-0.237761,-0.544210,-0.304370,-0.075846,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [314]:
X_train.drop(['High-density lipoprotein cholesterol ', 'Low-density lipoprotein cholesterol', 
                     'Aspartate transaminase'], axis = 1, inplace=True)

In [240]:
X_train_scaled.loc[abs(X_train_scaled['High-density lipoprotein cholesterol ']) < 1e-15, 'High-density lipoprotein cholesterol '] = 0

In [281]:
X_train_scaled.columns

Index(['Age', 'height', 'weight', 'Body Mass Index', 'Systolic blood pressure',
       'diastolic blood pressure', 'Fasting plasma glucose', 'Cholesterol',
       'Triglyceride', 'Alanine aminotransferase', 'Aspartate transaminase',
       'Blood urea nitrogen', 'Creatinine clearance rate',
       'family history of diabetes', 'Gender_1', 'Gender_2', 'site_0',
       'site_2', 'site_3', 'site_4', 'site_5', 'site_6', 'site_7', 'site_8',
       'site_9', 'site_10', 'site_11', 'site_12', 'site_13', 'site_14',
       'site_16', 'smoking status_1.0', 'smoking status_2.0',
       'smoking status_3.0', 'drinking status_1.0', 'drinking status_2.0',
       'drinking status_3.0'],
      dtype='object')

In [282]:
X_train_scaled.loc[:, :'site_0']

,Age,height,weight,Body Mass Index,Systolic blood pressure,diastolic blood pressure,Fasting plasma glucose,Cholesterol,Triglyceride,Alanine aminotransferase,Aspartate transaminase,Blood urea nitrogen,Creatinine clearance rate,family history of diabetes,Gender_1,Gender_2,site_0
0,0.861064,-0.357051,-0.050174,0.228036,-0.800857,-0.572627,-0.139734,-0.972282,0.010199,-0.381892,-4.558445e-16,0.309345,0.308560,0.0,1.0,0.0,0.0
1,-1.272461,-1.256006,0.309383,1.452767,-1.290207,-0.572627,0.334063,-0.658538,-0.602173,-0.539621,-1.120164e+00,-1.310327,-1.014147,0.0,0.0,1.0,0.0
2,-0.166189,-1.735448,-1.447544,-0.877208,-0.495013,-1.034272,1.150955,-0.456845,-0.621613,-0.472706,-9.020389e-01,-0.251311,-0.270535,0.0,0.0,1.0,0.0
3,-0.640306,1.261067,0.268524,-0.456021,0.850700,0.627649,-0.123396,-0.400819,0.049079,0.378072,3.168948e-01,1.003490,0.578366,0.0,1.0,0.0,0.0
4,-0.640306,1.380928,0.268524,-0.518751,0.300181,0.904636,-1.936895,0.349927,-0.213365,0.272920,-4.558445e-16,-0.874262,0.446754,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,2.362434,-0.596772,-0.630368,-0.399265,0.116675,1.181623,1.608414,0.932595,-0.164765,-0.329316,-4.558445e-16,0.656418,0.380947,0.0,1.0,0.0,0.0
99996,0.703025,1.320998,1.494287,0.915080,0.239012,0.442991,-0.254099,1.111878,0.787813,-0.377113,-4.558445e-16,0.104661,-0.138923,0.0,1.0,0.0,0.0
99997,-0.640306,0.781625,-0.393388,-0.966823,-1.840726,-1.772904,0.464766,-1.274822,-0.874338,-0.630434,-4.558445e-16,-0.411498,0.821850,0.0,1.0,0.0,0.0
99998,-0.798345,2.459674,2.556615,1.273538,1.707062,2.751215,-0.237761,-0.546486,-0.310567,0.320717,-4.558445e-16,-1.274730,0.223012,0.0,1.0,0.0,0.0


**Задание 2** Бэйзлайн – константное предсказание (0.5 балла).

Как понять, работает ли та или иная модель, если сравить метрику не с чем? Чтобы было с чем сравнивать, соберём простой бэйзлайн: предскажем всем клиентам одну и ту же вероятность заболеть в течение 5 лет. Какое значение надо предсказать, чтобы минимизировать кросс-энтропию? Оцените качество такого предсказания.

**Задание 3** Наивный байесовский классификатор (0.5 балла).

Предположим, что в каждом из классе признаки независимы и имеют нормальное распределение. Тогда подобрать параметры этого распределения поможет модель [Gaussian Naive Bayes](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html). Обучите эту модель. Оцените её качество.





In [15]:
naive_scores = cross_val_score(GaussianNB(), X_train, y_train['diabettes in 5 years'], scoring='neg_log_loss', cv=KFold(5))
-naive_scores.mean()

2.225858054736001

In [19]:
cross_val_score(model, X_train, y_train['diabettes in 5 years'], scoring='neg_log_loss', cv=KFold(5))

TypeError: Cannot clone object '<statsmodels.genmod.generalized_linear_model.GLM object at 0x7f700416c3d0>' (type <class 'statsmodels.genmod.generalized_linear_model.GLM'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.

In [17]:
from statsmodels import api as sm
from sklearn.metrics import log_loss

In [11]:
y_train

,diabettes in 5 years
Id,
0,0
1,0
2,0
3,0
4,0
...,...
99995,0
99996,0
99997,0


In [387]:
for name in X_train.columns:
    model = sm.GLM(y_train, X_train.drop(name, axis=1), \
               family=sm.families.Binomial(link=sm.genmod.families.links.logit()))
    result = model.fit()
    print(name, ":  ", log_loss(y_train, result.predict(X_train.drop(name, axis=1))))

Age :   0.06323827122999537
height :   0.061339113483283934
weight :   0.06135826404925487
Body Mass Index :   0.0612740777243939
Systolic blood pressure :   0.06121220998110131
diastolic blood pressure :   0.06120631092073917
Fasting plasma glucose :   0.07650555602970316
Cholesterol :   0.06121750597849511
Triglyceride :   0.06125115153386845
High-density lipoprotein cholesterol  :   0.06121440148658791
Low-density lipoprotein cholesterol :   0.06121749174620527
Alanine aminotransferase :   0.06154586673293504
Aspartate transaminase :   0.06120757471677538
Blood urea nitrogen :   0.06120794463399243
Creatinine clearance rate :   0.06142089002864047
family history of diabetes :   0.06129953650533578
Gender_1 :   0.06120631067926907
Gender_2 :   0.06120631067926907
site_0 :   0.061206310679269064
site_2 :   0.061206310679269064
site_3 :   0.061206310679269064


KeyboardInterrupt: 

In [18]:
model = sm.GLM(y_train, X_train, \
               family=sm.families.Binomial(link=sm.genmod.families.links.cauchy()))

In [444]:
result = model.fit()

In [445]:
log_loss(y_train, result.predict(X_train))

0.07108742185790139

### Best score: 0.061206310679269064 (0.6236) -> 0.06175

In [13]:
y_test = pd.read_csv('X_test.csv')

In [332]:
y_test = pd.DataFrame(replace_nan.fit_transform(y_test), columns=y_test.columns, index=y_test.index)

In [333]:
y_test = pd.get_dummies(y_test, columns=['Gender', 'site', 'smoking status', 'drinking status'])

In [338]:
y_test

,Age,height,weight,Body Mass Index,Systolic blood pressure,diastolic blood pressure,Fasting plasma glucose,Cholesterol,Triglyceride,High-density lipoprotein cholesterol,...,site_12.0,site_13.0,site_14.0,site_16.0,smoking status_1.0,smoking status_2.0,smoking status_3.0,drinking status_1.0,drinking status_2.0,drinking status_3.0
0,67.0,159.0,65.5,25.9,139.0,65.0,3.79,6.33,2.07,1.35,...,0,0,0,0,0,0,1,0,0,1
1,27.0,177.0,80.0,25.5,105.0,62.0,5.64,5.00,1.40,1.35,...,0,0,0,0,0,0,1,0,0,1
2,65.0,158.0,53.0,21.2,108.0,67.0,4.76,5.37,1.85,1.35,...,0,0,0,0,0,0,1,0,0,1
3,30.0,173.5,69.0,22.9,111.0,76.0,5.20,4.82,1.06,1.40,...,0,0,0,0,0,0,1,0,0,1
4,25.0,173.0,71.0,23.7,112.0,67.0,4.73,3.88,0.89,1.35,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111828,40.0,165.0,72.0,26.4,131.0,88.0,4.66,4.62,1.07,1.35,...,0,0,0,0,0,0,1,0,0,1
111829,30.0,152.8,45.8,19.6,97.0,69.0,4.50,3.29,0.53,1.35,...,0,0,0,0,0,0,1,0,0,1
111830,53.0,174.5,89.0,29.2,123.0,75.0,4.74,5.10,1.60,1.31,...,0,0,0,0,0,0,1,0,0,1
111831,32.0,169.0,70.0,24.5,119.0,66.0,5.04,4.09,1.08,1.35,...,0,0,0,0,0,0,1,0,0,1


In [174]:
y_test_scaled = y_test.copy()
y_test_scaled.loc[:, :'Creatinine clearance rate'] = \
    scaler.transform(y_test.astype(np.double).loc[:, :'Creatinine clearance rate'])

In [181]:
y_test_scaled

,Age,height,weight,Body Mass Index,Systolic blood pressure,diastolic blood pressure,Fasting plasma glucose,Cholesterol,Triglyceride,High-density lipoprotein cholesterol,...,site_12,site_13,site_14,site_16,smoking status_1.0,smoking status_2.0,smoking status_3.0,drinking status_1.0,drinking status_2.0,drinking status_3.0
0,1.967336,-0.896424,0.064231,0.795594,1.217712,-0.849614,-1.838868,1.817803,0.710051,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
1,-1.193442,1.261067,1.249135,0.676108,-0.862026,-1.126601,1.183631,0.327516,0.058800,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.809298,-1.016284,-0.957239,-0.608365,-0.678519,-0.664956,-0.254099,0.742107,0.496207,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.956384,0.841555,0.350242,-0.100550,-0.495013,0.166004,0.464766,0.125823,-0.271687,0.125202,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.351481,0.781625,0.513677,0.138422,-0.433844,-0.664956,-0.303112,-0.927462,-0.436930,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111828,-0.166189,-0.177260,0.595394,0.944952,0.728362,1.273952,-0.417477,-0.001587,-0.000777,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111829,-0.956384,-1.639560,-1.545605,-1.086309,-1.351376,-0.480298,-0.678882,-1.588566,-0.786856,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
111830,0.861064,0.961416,1.984592,1.781353,0.239012,0.073675,-0.286774,0.439568,0.253203,-0.267857,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
111831,-0.798345,0.302182,0.431959,0.377394,-0.005663,-0.757285,0.203361,-0.692153,-0.252246,0.005963,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [339]:
rew = result.predict(y_test)

In [340]:
data_fraim = pd.DataFrame()
data_fraim['Id'] = y_test.index
data_fraim['Prediction'] = rew

In [341]:
data_fraim.to_csv('ans5.csv', index=False)

**Задание 4** Дискриминантный анализ (1 балл).

![](https://scikit-learn.org/stable/_images/sphx_glr_plot_lda_qda_001.png)

Теперь избавимся от предположения условной независимости признаков относительно целевой переменной. Таким образом, ковариационные матрицы распределений классов не обязательно будут даигональны. Мы можем наложить дополнительное условие в виде [равенства ковариационных матриц](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis) всех классов или [не делать этого](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis).

Попробуйте оба варианта. Какой сработал лучше и чем это можно объяснить?


**Задание 5** Логистическая регрессия (1 балл).

Обучите модель логистической регрессии. Убедитесь, что модель сошлась. Удалось ли получить улучшение по сравнению с предыдущими моделями? Чем это можно объяснить?

**Задание 6** GLM – обобщённые линейные модели (1 балл).

Как вы знаете, логистическая регрессия является частным случаем обобщённой линейной модели $\mu(\mathbb E(y|X)) = Xw$, где функция связи $\mu(u) = \log(\frac{u}{1-u})$, и $y|X\sim Bernoulli$.

Учитывая, что целевая переменная бинарная, изменять класс распределений $y|X$ не имеет смысла. А вот изменить функцию связи можно.

Обучите обобщённые линейные модели, в качестве функции связи использовав по крайней мере 2 разные функции, отличные от logit, использующейся в логистической регрессии.

Реализацию GLM можно взять в пакете [statsmodels](https://www.statsmodels.org/stable/glm).



В этой части работы мы построим теперь вероятностную модель, учитывающую специфику задачи.

**Задание 7** Оценка Нельсона – Аалена (1 балл).

Поскольку изначально все участники исследования были здоровы, заболеть, скажем, через месяц у них очень мало шансов. Скорее всего, это значило бы ошибку при проведении анализов. А вот заболеть через несколько лет шансов уже больше.

Зависимость риска заболеть в данный момент времени $t$ при условии, что до момента $t$ человек оставался здоров, называется функцией риска ([hazard function](https://en.wikipedia.org/wiki/Survival_analysis#Hazard_function_and_cumulative_hazard_function)). В других задачах она позволяет определить, люди какого возраста наиболее подвержены заболеванию, или в какой момент эпидемии риск заразиться максимален.

Оцените, какова вероятность заболеть через $t$ лет после начала исследования, воспользовавшись оценками Нельсона – Аалена:

$$\hat H_{NA}(t) = \frac{d_t}{n_t},$$

Где $n_t$ – количество участников, остававшихся здоровыми и не прекративших участие в исследовании до года $t$, $d_t$ – количество участников, заболевших в год $t.$

Изобразите $\hat H_{NA}(t)$ на графике.

Оценка Нельсона – Аалена не персонализирована, т.е. для неё мы никак не использовали признаки пациентов.

Модель пропорциональных рисков Кокса (Cox Proportional Hazard Model, CPHM) моделирует отношение риска для пациента к среднему риску по всем пациентам для данного момента времени. Поскольку это отношение рисков – положительное число, его обычно моделируют как $e^{Xw}$, где $X$ – признаки пациентов, $w$ – обучаемый вектор весов. Таким образом,

$$\hat H_{Cox}(x, t) = \hat H_{NA}(t)e^{xw}.$$

Обратите внимание, что в линейной части CPHM не используют свободный член: в некотором смысле его роль играет $\hat H_{NA}(t)$.

Вероятность заболеть ровно в момент $t$ – это произведение вероятности не заболеть до момента $t$ на вероятность заболеть в момент $t$ при условии здоровья до момента $t$. Таким образом, функция правдоподобия имеет вид

$$L = \prod_{i=1}^N \left[(\hat H_{NA}(T_i)e^{x_iw})^{y_i}(1 - \hat H_{NA}(T_i)e^{x_iw})^{1-y_i}\prod_{t=0}^{T_i-1}\left(1-\hat H_{NA}(t)e^{x_iw}\right)\right],$$

где $T_i$ – момент последней записи, $i$-го пациента.
Максимизация правдоподобия эквивалентна минимизации «минус» нормированного логарифма правдоподобия.

$$\mathcal L = -\frac{1}{N}\log L = -\frac{1}{N}\sum_{i=1}^N\left[y_i\left(\log \hat H_{NA}(T_i) + x_iw\right) + (1-y_i)\left(\log (1-\hat H_{NA}(T_i) e^{x_iw})\right) + \sum_{t=0}^{T_i - 1}\log(1-\hat H_{NA}(t)e^{x_iw})\right]$$

$$\nabla_w\mathcal L = \frac{1}{N}\sum_{i=1}^N\left[-x_iy_i + (1-y_i)\frac{\hat H_{NA}(T_i)e^{x_iw}x_i}{1-\hat H_{NA}(T_i)e^{x_iw}} + \sum_{t=0}^{T_i-1}\frac{\hat H_{NA}(t)e^{x_iw}x_i}{1-\hat H_{NA}(t)e^{x_iw}}\right]$$

Минимизировать $\mathcal L$ можно градиентным спуском, итеративно применяя формулу

$$w:= w - \eta \nabla_w\mathcal L - \eta Cw$$

где $\eta > 0$ — размер шага (learning rate), $C\geq 0$ — коэффициент регуляризации.




**Задание 8** Cox Proportional Hazard Model (CPHM) (3 балла).

 Реализуйте градиентный спуск и примените его к модели CPHM.

В качестве критерия останова мы предлагаем использовать следующие условия:
 - евклидова норма разности текущего и нового векторов весов стала меньше, чем 1e-4
 - ограничение на число итераций (например, 10000)
 
Для начальной инициализации весов нужно сравнить следующие подходы:
 - нулевая начальная инициализация
 - случайная
 
Выполните следующие пункты и прокомментируйте полученные результаты:
- Рассмотрите как влияет размер шага на сходимость (попробуйте не менее 5-ти различных значений).
- Рассмотрите регуляризованную модель (не менее 5-ти различных коэффициентов регуляризации), которая описана выше, а также модель без регуляризатора. Сравните, влияет ли наличие регуляризации на скорость сходимости и качество (под качеством во всех случаях подразумевается значение исходного, нерегуляризованного функционала).
- Исследуйте качество оптимизируемого функционала в зависимости от номера итерации (при правильной реализации и подходящем размере шага он должен убывать).
- Влияет ли выбор начальной инициализации весов на скорость и качество?

В каждом пункте требуется построить необходимые графики скорости/качества и дать исчерпывающие выводы.


Метод градиентного спуска может быть весьма трудозатратен в случае большого размера обучающей выборки. Поэтому обычно используют метод стохастического градиентного спуска, где на каждой итерации выбирается случайный объект из обучающей выборки и обновление весов происходит сразу по этому объекту.

**Задание 9** SGD (1 балл)

Реализуйте метод стохастического градиентного спуска (sgd). В этом случае вы можете выбрать наиболее удачный функционал, исходя из предыдущего пункта (с регуляризацией, без), а также схему начальной инициализации весов.

Сравните рассмотренные методы (градиентный спуск и sgd) между собой с точки зрения скорости сходимости и качества.

Посмотрите как влияет размер шага на сходимость (попробуйте 4-5 различных значений)
Исследуйте качество оптимизируемого функционала в зависимости от номера итерации
Выберите лучший размер шага.

В каждом пункте сделайте исчерпывающие выводы, подкреплённые графиками.

Между обновлением вектора весов по всей выборке и на одном объекте есть промежуточный подход — выбирать некоторое случайное подмножество объектов и обновлять веса по нему. Такой подход называется mini-batch. Мы не будем реализовывать этот подход в данной работе, однако иногда его бывает осмысленно использовать на практике. 

Один из недостатков sgd состоит в том, что он может не доходить до локального оптимального решения, а осциллировать в окрестности. 

![](http://sebastianruder.com/content/images/2015/12/without_momentum.gif)

Для решения этой проблемы существуют методы, позволяющие устранить этот недостаток, а также ускорить сходимость. Рассмотрим некоторые из них.

![](http://nghenglim.github.io/images/2015061300.png)

### Momentum

Этот метод позволяет направить sgd в нужной размерности и уменьшить осцилляцию. 

В общем случае он будет выглядеть следующим образом: 

$$ v_t = \gamma v_{t - 1} + \eta \nabla_{w}{J(w)}$$
$$ w = w - v_t$$

где

 - $w$ — вектор параметров
 - $J$ — оптимизируемый функционал
 - $\gamma$ — momentum term (обычно выбирается 0.9)
 
### Adagrad 

Одной из сложностей является выбор размера шага (*learning rate*). Основное отличие данного метода от SGD состоит в том что размер шага определяется для каждого параметра индивидуально. Этот метод хорошо работает с разреженным данным большого объема. 

Обозначим градиент по параметру $w_i$ на итерации $t$ как $g_{t,i} = \nabla_{w}J(w_i)$. 

В случае sgd обновление параметра $w_i$ будет выглядеть следующим образом:

$$ w_{t+1, i} = w_{t, i} - \eta \cdot g_{t,i}$$

А в случае Adagrad общий шаг $\eta$ нормируется на посчитанные ранее градиенты для данного параметра:

$$ w_{t+1, i} = w_{t, i} - \dfrac{\eta}{\sqrt{G_{t,ii} + \varepsilon}} \cdot g_{t,i}$$

где $G_t$ — диагональная матрица, где каждый диагональный элемент $i,i$ — сумма квадратов градиентов для $w_{i}$ до $t$-ой итерации. $\varepsilon$ — гиперпараметр, позволяющий избежать деления на 0 (обычно выбирается около *1e-8*).

Так как матрица $G_t$ диагональна, в векторном виде это будет выглядеть следующим образом (здесь $\odot$ — матричное умножение):

$$ w_{t+1} = w_{t} - \dfrac{\eta}{\sqrt{G_t + \varepsilon}} \odot g_t $$

### Adadelta

Adadelta, в отличии от Adagrad, рассматривает не все предыдущие значения градиентов, а только последние $k$. Кроме того, сумма градиентов определяется как уменьшающееся среднеее всех предыдущих квадратов градиентов. Текущее среднее $E[g^2]_t$ на итерации $t$ будет вглядеть следующим образом:

$$ E[g^2]_t = \gamma E[g^2]_{t-1} + (1-\gamma)g_t^2 $$

здесь $\gamma$ аналогична гиперпараметру из метода Momentum.

Тогда обновление весов можно записать следующим образом:

$$ w_{t+1} = w_{t} - \dfrac{\eta}{\sqrt{E[g^2]_t + \varepsilon}} g_t $$ 

Перепишем это немного по-другому:

$$ w_{t+1} = w_{t} + \Delta w_t$$ 
$$\Delta w_t = - \dfrac{\eta}{\sqrt{E[g^2]_t + \varepsilon}} g_t $$ 

Аналогично среднему для градиентов определим среднее для параметров $w$:

$$ E[\Delta w^2]_t = \gamma E[\Delta w^2]_{t-1} + (1-\gamma)\Delta w^2 $$

Введем обозначение $RMS[p]_t = \sqrt{E[p]_t + \varepsilon}$

Тогда Adadelta выглядит следующим образом:

$$\Delta w_t = - \dfrac{RMS[\Delta w^2]}{RMS[ga^2]} g_t $$ 
$$ w_{t+1} = w_{t} + \Delta w_t$$ 


Более подробно об этих и других способах оптимизации можно прочитать:
 - [здесь](http://sebastianruder.com/optimizing-gradient-descent/index.html#gradientdescentoptimizationalgorithms) очень хорошее описание различных способов оптимизации, в этом задании мы опираемся на терминологию из данной статьи
 - статья про [momentum](https://pdfs.semanticscholar.org/97da/c94ffd7a7ac09a4218848300cc7e98569d77.pdf)
 - оригинальная статья про [adagrad](http://www.jmlr.org/papers/volume12/duchi11a/duchi11a.pdf)
 - оригинальная статья про [adadelta](http://arxiv.org/pdf/1212.5701v1.pdf)
 - википедия про [momentum](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#Momentum) и [adagrad](https://en.wikipedia.org/wiki/Stochastic_gradient_descent#AdaGrad)
 - [визуализация](http://imgur.com/a/Hqolp) разных способов оптимизации

**Задание 10** Реализуйте метод оптимизации *Momentum* (0.5 балла) и 

**Задание 11** Реализуйте один из *Adagrad*/*Adadelta* (1 балл).

В заданиях 10, 11:
- Сравните оба метода с классическим sgd с точки зрения скорости сходимости.
- Посмотрите как значение гиперпараметра $\gamma$ влияет на скорость сходимости и качество в методе *Momentum*.

Постройте графики и опишите полученные результаты.

Дало ли преимущество использование адаптивного шага в методе *Adagrad*/*Adadelta*?

**Задание 12** Предсказание с помощью CPHM, интерфейс sklearn (1 балл)


Чтобы сравнить CPHM с предыдущими моделями, надо научиться предсказывать целевую переменную. С помощью значений персональной функции риска $\hat H_{Cox}(x, t)$ можно выразить вероятность того, что человек в течение 5 лет не заболеет при условии, что он не прекратит участие в эксперементе по другим причинам. Вероятность того, что человек не прекратит участие в эксперименте, оставаясь здоровым, также можно оценить из данных.

Реализуйте модель CPHM, унаследовав класс модели от [BaseEstimator](https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator). Модель должна поддерживать методы fit, predict, predict_proba. Это вам позволит в следующем задании её откалибровать. При тестировании модели следует передать то время, через которое вас интересует состояние пациента (в случае конкурса, 5 лет).




**Задание 13** Калибровка классификатора (2 балла)

Сравнительно простой способ уточнить оценки вероятности и уменьшить logloss -- откалибровать модель. Вообще говоря, сравнивать между собой некалиброванные модели по logloss некорректно: хорошая модель с нарушенной калибровкой может иметь сколь угодно большое значение logloss.

Идея калибровки состоит в том, чтобы подобрать простое преобразование, которое превратит выходы моделей в вероятности принадлежности классам.

Есть несколько известных методов калибровки:
 - Калибровка Платта.
 - Изотоническая регрессия.

Обратите внимание, что преобразование необходимо обучать на отложенной выборке (то есть классификатор и калибровка должны обучаться на разных подмножествах данных), иначе можно переобучиться. Калибровку можно применять к любым классификаторам (где это разумно и необходимо), особенно к тем, которые не оптимизируют logloss явно.

Для калибровки классификатора в sklearn возможны два подхода:
 - взять уже обученный классификатор и откалибровать его на отложенной выборке
 - откалибровать по кросс-валидации: калибровочному классификатору передается вся обучающая выборка, которая внутри разбивается на обучающую и калибровочную, после чего происходит усреднение вероятностей по фолдам.
 
Подробнее об этом можно прочитать в [документации](http://scikit-learn.org/stable/modules/generated/sklearn.calibration.CalibratedClassifierCV.html#sklearn.calibration.CalibratedClassifierCV). Также [здесь](https://jmetzen.github.io/2015-04-14/calibration.html) можно узнать подробности о калибровке в sklearn от автора.

Используйте оба описанных выше подхода (калибровку Платта и изотоническую регрессию) для калибровки моделей, построенных в лабораторной работе. Для каждой модели постройте график, на котором будут изображены [калибровочные кривые](http://scikit-learn.org/stable/modules/generated/sklearn.calibration.calibration_curve.html): идеальная, исходного классификатора, а также для каждого из методов. Калибровочная кривая строится путем упорядочения всех объектов по предсказанному значению, которые разбиваются на бины. По оси OX откладывается среднее предсказанное значение вероятности по бину, а по OY — доля положительных примеров. В случае идеальных вероятностей это будет прямая.
 


Выполнив задания, не забудьте отправить решение в [конкурс](https://www.kaggle.com/c/competition-2-yandex-shad-spring-2021)

**Выводы** (0.5 балла)

- Какие есть достоинства и недостатки у рассмотренных в лабораторной работе моделей?
- Какие модели сильно улучшили свой logloss после калибровки, а какие - нет? Почему так произошло?
- Какие из рассмотренных методов могут обучиться на маленьких датасетах? А какие применимы для очень больших датасетов? Какие проблемы могут возникнуть при применении к большим датасетам и как их можно решить?
